# Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)

from sklearn.metrics import mean_squared_log_error

from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge

# Read Data

In [2]:
economic_data = pd.read_csv('../01-Data/economic_data.csv',  parse_dates=['Date'])

In [3]:
economic_data.head()

,Date,Oil_mean,Selic_mean,Dollar_mean,Gold_mean,Oil_median,Selic_median,Dollar_median,Gold_median,Oil_std,Selic_std,Dollar_std,Gold_std
0,2004-05-09,39.297143,16.0,3.014014,37.064286,39.41,16.0,2.989100,36.90,0.578179,0.0,0.059211,0.414151
1,2004-05-16,40.687143,16.0,3.113114,37.492857,40.94,16.0,3.113267,37.45,0.943112,0.0,0.011204,0.332757
2,2004-05-23,40.867143,16.0,3.158643,38.290000,40.92,16.0,3.180500,38.30,0.664057,0.0,0.042535,0.569077
3,2004-05-30,40.693214,16.0,3.137500,38.778571,40.60,16.0,3.151600,38.70,0.939225,0.0,0.031106,0.305007
4,2004-06-06,39.843214,16.0,3.133371,38.828571,39.29,16.0,3.129400,38.70,1.591660,0.0,0.012524,0.541322


In [4]:
data = pd.read_csv('../01-Data/2004-2019.tsv', sep='\t', parse_dates=['DATA INICIAL', 'DATA FINAL'])

In [5]:
data.head()

,Unnamed: 0,DATA INICIAL,DATA FINAL,REGIÃO,ESTADO,PRODUTO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,MARGEM MÉDIA REVENDA,COEF DE VARIAÇÃO REVENDA,PREÇO MÉDIO DISTRIBUIÇÃO,DESVIO PADRÃO DISTRIBUIÇÃO,PREÇO MÍNIMO DISTRIBUIÇÃO,PREÇO MÁXIMO DISTRIBUIÇÃO,COEF DE VARIAÇÃO DISTRIBUIÇÃO,MÊS,ANO
0,0,2004-05-09,2004-05-15,CENTRO OESTE,DISTRITO FEDERAL,ETANOL HIDRATADO,127,R$/l,1.288,0.016,1.19,1.350,0.463,0.012,0.825,0.11,0.4201,0.9666,0.133,5,2004
1,1,2004-05-09,2004-05-15,CENTRO OESTE,GOIAS,ETANOL HIDRATADO,387,R$/l,1.162,0.114,0.89,1.449,0.399,0.098,0.763,0.088,0.5013,1.05,0.115,5,2004
2,2,2004-05-09,2004-05-15,CENTRO OESTE,MATO GROSSO,ETANOL HIDRATADO,192,R$/l,1.389,0.097,1.18,1.760,0.419,0.070,0.97,0.095,0.5614,1.161,0.098,5,2004
3,3,2004-05-09,2004-05-15,CENTRO OESTE,MATO GROSSO DO SUL,ETANOL HIDRATADO,162,R$/l,1.262,0.070,1.09,1.509,0.432,0.055,0.83,0.119,0.5991,1.22242,0.143,5,2004
4,4,2004-05-09,2004-05-15,NORDESTE,ALAGOAS,ETANOL HIDRATADO,103,R$/l,1.181,0.078,1.05,1.400,0.24,0.066,0.941,0.077,0.7441,1.0317,0.082,5,2004


# Translate column names to English

In [6]:
data.columns

Index(['Unnamed: 0', 'DATA INICIAL', 'DATA FINAL', 'REGIÃO', 'ESTADO',
       'PRODUTO', 'NÚMERO DE POSTOS PESQUISADOS', 'UNIDADE DE MEDIDA',
       'PREÇO MÉDIO REVENDA', 'DESVIO PADRÃO REVENDA', 'PREÇO MÍNIMO REVENDA',
       'PREÇO MÁXIMO REVENDA', 'MARGEM MÉDIA REVENDA',
       'COEF DE VARIAÇÃO REVENDA', 'PREÇO MÉDIO DISTRIBUIÇÃO',
       'DESVIO PADRÃO DISTRIBUIÇÃO', 'PREÇO MÍNIMO DISTRIBUIÇÃO',
       'PREÇO MÁXIMO DISTRIBUIÇÃO', 'COEF DE VARIAÇÃO DISTRIBUIÇÃO', 'MÊS',
       'ANO'],
      dtype='object')

In [7]:
data.columns = [
                    "Unnamed:_0", 
                    "Analysis_Date", #DATA INICIAL
                    "Last_Day_of_Analyses_of_Week", #DATA FINAL
                    "Macroregion", #REGIÃO
                    "State", #ESTADO
                    "Product", #PRODUTO
                    "No_of_Gas_Stations_Analyzed", #NÚMERO DE POSTOS PESQUISADOS
                    "Measurement_Unit", #UNIDADE DE MEDIDA
                    "Mean_Price", #PREÇO MÉDIO REVENDA
                    "Std_Dev", #MARGEM MÉDIA REVENDA
                    "Min_Price", #DESVIO PADRÃO REVENDA
                    "Max_Price", #PREÇO MÁXIMO REVENDA
                    "Mean_Price_Margin", #MARGEM MÉDIA REVENDA
                    "Coefficient_of_variation", #COEF DE VARIAÇÃO REVENDA
                    "Mean_Dist_Price", #PREÇO MÉDIO DISTRIBUIÇÃO
                    "Distribution_Std_Dev", #DESVIO PADRÃO DISTRIBUIÇÃO
                    "Distribution_Min_Price", #PREÇO MÍNIMO DISTRIBUIÇÃO
                    "Distribution_Max_Price", #PREÇO MÁXIMO DISTRIBUIÇÃO
                    "Distribution_Coefficient_of_Variation", #COEF DE VARIAÇÃO DISTRIBUIÇÃO
                    "Month", #MÊS
                    "Year" #ANO
]

In [8]:
data.columns

Index(['Unnamed:_0', 'Analysis_Date', 'Last_Day_of_Analyses_of_Week',
       'Macroregion', 'State', 'Product', 'No_of_Gas_Stations_Analyzed',
       'Measurement_Unit', 'Mean_Price', 'Std_Dev', 'Min_Price', 'Max_Price',
       'Mean_Price_Margin', 'Coefficient_of_variation', 'Mean_Dist_Price',
       'Distribution_Std_Dev', 'Distribution_Min_Price',
       'Distribution_Max_Price', 'Distribution_Coefficient_of_Variation',
       'Month', 'Year'],
      dtype='object')

# Join the Economic Data with the oil data

In [9]:
data = data.merge(economic_data, left_on='Analysis_Date', right_on='Date')

In [10]:
data.sample(10)

,Unnamed:_0,Analysis_Date,Last_Day_of_Analyses_of_Week,Macroregion,State,Product,No_of_Gas_Stations_Analyzed,Measurement_Unit,Mean_Price,Std_Dev,Min_Price,Max_Price,Mean_Price_Margin,Coefficient_of_variation,Mean_Dist_Price,Distribution_Std_Dev,Distribution_Min_Price,Distribution_Max_Price,Distribution_Coefficient_of_Variation,Month,Year,Date,Oil_mean,Selic_mean,Dollar_mean,Gold_mean,Oil_median,Selic_median,Dollar_median,Gold_median,Oil_std,Selic_std,Dollar_std,Gold_std
19375,4223,2007-05-20,2007-05-26,NORTE,PARA,ETANOL HIDRATADO,94,R$/l,2.276,0.113,2.049,2.620,0.295,0.050,1.981,0.094,1.636,2.2089,0.047,5,2007,2007-05-20,64.174286,12.50,1.969400,45.407143,64.830000,12.50,1.957800,45.500000,1.375207,0.0,0.023571,0.276194
3920,36687,2004-12-12,2004-12-18,NORDESTE,PIAUI,GNV,1,R$/m3,1.399,0.000,1.399,1.399,-,0.000,-,-,-,-,-,12,2004,2004-12-12,41.617143,17.25,2.761300,39.128571,41.510000,17.25,2.773600,39.000000,0.862429,0.0,0.026127,0.401930
59699,59699,2013-06-30,2013-07-06,SUDESTE,ESPIRITO SANTO,GNV,14,R$/m3,1.929,0.201,1.790,2.430,0.656,0.104,1.273,0.054,1.2145,1.2904,0.042,6,2013,2013-06-30,96.207143,8.00,2.215571,89.028571,96.360000,8.00,2.218000,88.600000,0.941739,0.0,0.021198,2.214132
1635,12422,2004-08-08,2004-08-14,NORDESTE,MARANHAO,GASOLINA COMUM,189,R$/l,2.182,0.144,1.810,2.480,0.367,0.066,1.815,0.043,1.7307,1.9496,0.024,8,2004,2004-08-08,43.977143,16.00,3.050714,38.567143,44.130000,16.00,3.048100,38.700000,0.603282,0.0,0.007558,0.356113
28129,30245,2008-09-21,2008-09-27,NORTE,AMAZONAS,GLP,183,R$/13Kg,30.289,2.910,26.000,42.000,5.11,0.096,25.179,1.714,23.09,29.53,0.068,9,2008,2008-09-21,101.801429,13.75,1.844043,51.271429,97.500000,13.75,1.839000,51.333333,8.885891,0.0,0.036459,2.903155
13992,46595,2006-07-16,2006-07-22,SUL,RIO GRANDE DO SUL,ÓLEO DIESEL,617,R$/l,1.960,0.059,1.790,2.155,0.239,0.030,1.721,0.03,1.6105,1.8219,0.017,7,2006,2006-07-16,75.505714,15.25,2.199429,46.660000,76.066667,15.25,2.209800,47.133333,1.334453,0.0,0.015836,1.327572
99860,99860,2018-08-05,2018-08-11,CENTRO OESTE,GOIAS,GASOLINA COMUM,228,R$/l,4.684,0.151,4.299,4.999,0.545,0.032,4.139,0.074,4.02,4.28,0.018,8,2018,2018-08-05,69.258571,6.50,3.734486,145.171429,68.836667,6.50,3.720367,146.100000,0.961345,0.0,0.020304,1.592393
9017,45503,2005-10-09,2005-10-15,NORTE,ACRE,ÓLEO DIESEL,42,R$/l,2.252,0.144,2.130,2.600,0.329,0.064,1.923,0.108,1.771,2.1465,0.056,10,2005,2005-10-09,62.542857,19.50,2.256029,34.157143,61.810000,19.50,2.254000,34.266667,1.516716,0.0,0.018377,0.405044
21752,16791,2007-09-30,2007-10-06,CENTRO OESTE,MATO GROSSO DO SUL,GASOLINA COMUM,136,R$/l,2.667,0.086,2.544,2.840,0.421,0.032,2.246,0.049,2.1,2.439,0.022,9,2007,2007-09-30,81.495714,11.25,1.845700,43.807143,81.200000,11.25,1.840100,43.833333,0.916951,0.0,0.015814,0.178434
85582,85582,2016-10-09,2016-10-15,SUDESTE,SAO PAULO,GNV,61,R$/m3,2.058,0.174,1.799,2.649,0.583,0.085,1.475,0.164,1.3088,1.7645,0.111,10,2016,2016-10-09,49.562857,14.25,3.222300,130.657143,49.760000,14.25,3.219700,129.500000,0.619158,0.0,0.010396,2.130442


# Train and Validation Split (Simple Holdout)

In [378]:
data_train = data[data['Last_Day_of_Analyses_of_Week'] < '2011-01-01']
data_valid = data[data['Last_Day_of_Analyses_of_Week'] >= '2011-01-01']

data_train.shape, data_valid.shape

((42514, 34), (64309, 34))

# New DataFrame for Train and Validation (Index: original Data)

In [379]:
df_train = pd.DataFrame(index=data_train.index)
df_valid  = pd.DataFrame(index=data_valid.index)

# Target

## First Difference of Average Resale Price

In [380]:
df_train['diff_Mean_Price'] = data_train.groupby(['Product', 'State'])['Mean_Price'].apply(lambda row: row.diff().shift(-1))
df_valid['diff_Mean_Price'] = data_valid.groupby(['Product', 'State'])['Mean_Price'].apply(lambda row: row.diff().shift(-1))

# Features

## Current Mean Price

In [381]:
df_train['Current_Mean_Price'] = data_train['Mean_Price']
df_valid['Current_Mean_Price'] = data_valid['Mean_Price']

## Seasonality

In [382]:
df_train['month'] = data_train['Last_Day_of_Analyses_of_Week'].dt.month
df_train['day'] = data_train['Last_Day_of_Analyses_of_Week'].dt.day
#df_train['weekday'] = data_train['Last_Day_of_Analyses_of_Week'].dt.weekday
df_train['dayofyear'] = data_train['Last_Day_of_Analyses_of_Week'].dt.dayofyear

df_valid['month'] = data_valid['Last_Day_of_Analyses_of_Week'].dt.month
df_valid['day'] = data_valid['Last_Day_of_Analyses_of_Week'].dt.day
#df_valid['weekday'] = data_valid['Last_Day_of_Analyses_of_Week'].dt.weekday
df_valid['dayofyear'] = data_valid['Last_Day_of_Analyses_of_Week'].dt.dayofyear

## Movel Average

In [383]:
roll = data_train.groupby(['Product', 'State'])['Mean_Price'].rolling(4).mean()
roll.reset_index(level=[0,1], drop=True).head()

13         NaN
136        NaN
259        NaN
382    1.80525
505    1.80550
Name: Mean_Price, dtype: float64

In [384]:
df_train['Movel_Average_Mean_Price_4_weeks'] = data_train.groupby(['Product', 'State'])['Mean_Price'].rolling(4).mean().reset_index(level=[0,1], drop=True)
df_valid['Movel_Average_Mean_Price_4_weeks'] = data_valid.groupby(['Product', 'State'])['Mean_Price'].rolling(4).mean().reset_index(level=[0,1], drop=True)

## Economics Features

In [385]:
### Oil_mean	Selic_mean	Dollar_mean	Gold_mean

In [386]:
#mean

# df_train['Oil_mean'] = data_train['Oil_mean']
# df_train['Selic_mean'] = data_train['Selic_mean']
df_train['Oil_mean/Selic_mean'] = data_train['Oil_mean'] / data_train['Selic_mean']
# df_train['Dollar_mean'] = data_train['Dollar_mean']
# df_train['Gold_mean'] = data_train['Gold_mean']

# df_valid['Oil_mean'] = data_valid['Oil_mean']
# df_valid['Selic_mean'] = data_valid['Selic_mean']
df_valid['Oil_mean/Selic_mean'] = data_valid['Oil_mean'] / data_valid['Selic_mean']
# df_valid['Dollar_mean'] = data_valid['Dollar_mean']
# df_valid['Gold_mean'] = data_valid['Gold_mean']

In [387]:
#median

# df_train['Oil_median'] = data_train['Oil_median']
# df_train['Selic_median'] = data_train['Selic_median']
# df_train['Dollar_median'] = data_train['Dollar_median']
# df_train['Gold_median'] = data_train['Gold_median']

# df_valid['Oil_median'] = data_valid['Oil_median']
# df_valid['Selic_median'] = data_valid['Selic_median']
# df_valid['Dollar_median'] = data_valid['Dollar_median']
# df_valid['Gold_median'] = data_valid['Gold_median']

In [388]:
#std

# df_train['Oil_std'] = data_train['Oil_std']
# df_train['Selic_std'] = data_train['Selic_std']
# df_train['Dollar_std'] = data_train['Dollar_std']
# df_train['Gold_std'] = data_train['Gold_std']

# df_valid['Oil_std'] = data_valid['Oil_std']
# df_valid['Selic_std'] = data_valid['Selic_std']
# df_valid['Dollar_std'] = data_valid['Dollar_std']
# df_valid['Gold_std'] = data_valid['Gold_std']

In [389]:
df_train.isnull().sum()

diff_Mean_Price                     134
Current_Mean_Price                    0
month                                 0
day                                   0
dayofyear                             0
Movel_Average_Mean_Price_4_weeks    401
Oil_mean/Selic_mean                   0
dtype: int64

In [390]:
###### Drop NA

############# >>>>>>>>>>>>>>>>> Interpolate?


df_train = df_train.dropna()
df_valid = df_valid.dropna()

## X, y Train and Validation Split

In [391]:
Xtr, ytr = df_train.drop(['diff_Mean_Price'], axis=1), df_train['diff_Mean_Price']
Xval, yval = df_valid.drop(['diff_Mean_Price'], axis=1), df_valid['diff_Mean_Price']

# Model

## RandomForestRegressor

In [365]:
mdl = RandomForestRegressor(n_jobs=-1, random_state=0, n_estimators=500)
mdl.fit(Xtr, ytr)
p = mdl.predict(Xval)

#### Metric

In [367]:
p_final = Xval['Current_Mean_Price'] + p
yval_final = Xval['Current_Mean_Price'] + yval

np.sqrt(mean_squared_log_error(yval_final, p_final)) * 100

1.265551653411377

## LGBMRegressor

In [392]:
mdl = LGBMRegressor(num_leaves=2, min_data_in_leaf=250, n_jobs=-1, random_state=0, n_estimators=500)
mdl.fit(Xtr, ytr)
p = mdl.predict(Xval)

[LightGBM] [Warning] min_data_in_leaf is set=250, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=250


#### Metric

In [393]:
p_final = Xval['Current_Mean_Price'] + p
yval_final = Xval['Current_Mean_Price'] + yval

np.sqrt(mean_squared_log_error(yval_final, p_final)) * 100

1.2448081487292426

## LinearRegression

In [394]:
mdl = LinearRegression()
mdl.fit(Xtr, ytr)
p = mdl.predict(Xval)

#### Metric

In [395]:
p_final = Xval['Current_Mean_Price'] + p
yval_final = Xval['Current_Mean_Price'] + yval

np.sqrt(mean_squared_log_error(yval_final, p_final)) * 100

1.2282596740482903

## Lasso

In [396]:
mdl = Lasso()
mdl.fit(Xtr, ytr)
p = mdl.predict(Xval)

#### Metric

In [397]:
p_final = Xval['Current_Mean_Price'] + p
yval_final = Xval['Current_Mean_Price'] + yval

np.sqrt(mean_squared_log_error(yval_final, p_final)) * 100

1.242464532053409

## Ridge

In [398]:
mdl = Ridge()
mdl.fit(Xtr, ytr)
p = mdl.predict(Xval)

#### Metric

In [399]:
p_final = Xval['Current_Mean_Price'] + p
yval_final = Xval['Current_Mean_Price'] + yval

np.sqrt(mean_squared_log_error(yval_final, p_final)) * 100

1.2281631620398965

## BayesianRidge

In [400]:
mdl = BayesianRidge()
mdl.fit(Xtr, ytr)
p = mdl.predict(Xval)

#### Metric

In [401]:
p_final = Xval['Current_Mean_Price'] + p
yval_final = Xval['Current_Mean_Price'] + yval

np.sqrt(mean_squared_log_error(yval_final, p_final)) * 100

1.228178084175797